In [1]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from classic_patterns import classic_ml_data
from indicadores import *


In [2]:
import labeling as lb

returns_series
0      -0.002515
1      -0.081723
2      -0.134312
3      -0.175470
4      -0.019431
          ...   
3555    0.061436
3556    0.009204
3557    0.030082
3558   -0.007300
3559    0.009767
Length: 3560, dtype: float64
returns_series
0            NaN
1       0.056009
2       0.063704
3       0.068316
4       0.079537
          ...   
3555    0.046376
3556    0.038243
3557    0.032097
3558    0.030072
3559    0.024568
Length: 3560, dtype: float64
[nan, 0.05600866566138478, 0.06370374937981206, 0.0683157993062677, 0.07953702477780306, 0.11950940410898792, 0.09549885580369279, 0.07687153965758123, 0.07453256179382457, 0.07340546021232042]
returns_series[:10]
[0.051248915737722466, 0.044683419283526246, 0.07128063674707072, 0.058206728659875334, 0.05167097695847133, 0.0421810724868715, 0.03762627047562024, 0.032900063218733865, 0.02767061092564653]
returns_series[11:20]
[0.021342047337842753, 0.03557470353318286, 0.03567833504873975, 0.040650824174106634, 0.04012125964815533, 0

## Juntando os dados
Pegando os dados

In [3]:
# Convertendo para numpy arrays, caso ainda não estejam
ml_tsla_data = np.array(classic_ml_data)  
label_tsla_data = np.array(lb.label_tsla_data).ravel()

# Divide os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(ml_tsla_data, label_tsla_data, test_size=0.2, random_state=42)

## Fazendo a Rede Neural

In [8]:
def mlp(data, hidden_layers=(100, 100, 100), activation='logistic', 
        solver='adam', max_iter=500, random_state=42):
    """
    Treina um modelo de rede neural MLP e retorna as previsões e o relatório de classificação.

    Parâmetros:
    - data: DataFrame contendo os dados de entrada.
    - hidden_layers: Tupla com o tamanho das camadas ocultas.
    - activation: Função de ativação a ser usada.
    - solver: Algoritmo de otimização a ser usado.
    - max_iter: Número máximo de iterações.
    - random_state: Semente para a geração de números aleatórios.

    Retorna:
    - y_pred_mlp: Previsões das classes no conjunto de teste.
    - report: Relatório de classificação.
    """
    
    # 1. Agregar os indicadores
    indicators_df = agg_indicators(data)

    # 2. Normalizar os indicadores
    normalized_data = normalize_indicators(indicators_df)

    # 3. Separar as variáveis independentes (X) e dependentes (y)
    X = normalized_data.drop(columns=['Adj Close'])  # Variáveis independentes
    y = normalized_data['Adj Close']  # Variável dependente

    # 4. Dividir os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

    # 5. Define a rede neural com múltiplas camadas
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layers, activation=activation,
                        solver=solver, max_iter=max_iter, random_state=random_state)

    # 6. Treina a rede neural
    mlp.fit(X_train, y_train)

    # 7. Faz previsões de classe
    y_pred_mlp = mlp.predict(X_test)

    # 8. Exibe o relatório de classificação para o MLP
    report = classification_report(y_test, y_pred_mlp)
    print(report)

    return y_pred_mlp, report

## Fazendo o Modelo Random Forest

In [5]:
def random_forest(data, target_column, n_estimators=100, max_depth=None, random_state=42):
    """
    Treina um modelo de Random Forest e retorna as previsões e o relatório de classificação.

    Parâmetros:
    - data: DataFrame com as features e a variável alvo.
    - target_column: Nome da coluna alvo no DataFrame.
    - n_estimators: Número de árvores na floresta.
    - max_depth: Profundidade máxima das árvores (None para ilimitado).
    - random_state: Semente para a geração de números aleatórios.

    Retorna:
    - y_pred_rf: Previsões das classes no conjunto de teste.
    - report: Relatório de classificação.
    """
    
    # 1. Aplica as funções de pré-processamento
    data = agg_indicators(data)  # Ajuste conforme necessário
    normalized_data = normalize_indicators(data)  # Ajuste conforme necessário

    # 2. Define X e y
    X = normalized_data.drop(columns=[target_column])  # Variáveis independentes
    y = normalized_data[target_column]  # Variável dependente

    # 3. Divide os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

    # 4. Define o modelo Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, 
                                 random_state=random_state)

    # 5. Treina o modelo Random Forest
    rf.fit(X_train, y_train)

    # 6. Faz previsões de classe
    y_pred_rf = rf.predict(X_test)

    # 7. Exibe o relatório de classificação para o Random Forest
    report = classification_report(y_test, y_pred_rf)
    print(report)

    return y_pred_rf, report

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         9
         0.0       0.47      0.10      0.17       357
         1.0       0.48      0.88      0.63       347

    accuracy                           0.48       713
   macro avg       0.32      0.33      0.27       713
weighted avg       0.47      0.48      0.39       713



c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

## Fazendo o Modelo Gradient Boosting

In [6]:
def gradient_boosting(data, target_column, random_state=42):
    """
    Treina um modelo de Gradient Boosting e retorna as previsões e o relatório de classificação.

    Parâmetros:
    - data: DataFrame com as features e a variável alvo.
    - target_column: Nome da coluna alvo no DataFrame.
    - random_state: Semente para a geração de números aleatórios.

    Retorna:
    - y_pred_gb: Previsões das classes no conjunto de teste.
    - report: Relatório de classificação.
    """
    
    # 1. Aplica as funções de pré-processamento
    data = agg_indicators(data)  # Ajuste conforme necessário
    normalized_data = normalize_indicators(data)  # Ajuste conforme necessário

    # 2. Define X e y
    X = normalized_data.drop(columns=[target_column])  # Variáveis independentes
    y = normalized_data[target_column]  # Variável dependente

    # 3. Divide os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

    # 4. Define o modelo Gradient Boosting
    gb_model = GradientBoostingClassifier(random_state=random_state)

    # 5. Treina o modelo Gradient Boosting
    gb_model.fit(X_train, y_train)

    # 6. Faz previsões de classe
    y_pred_gb = gb_model.predict(X_test)

    # 7. Exibe o relatório de classificação para o Gradient Boosting
    report = classification_report(y_test, y_pred_gb)
    print(report)

    return y_pred_gb, report

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         9
         0.0       0.45      0.07      0.12       357
         1.0       0.48      0.92      0.63       347

    accuracy                           0.48       713
   macro avg       0.31      0.33      0.25       713
weighted avg       0.46      0.48      0.37       713



c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [57]:
print(y_pred_probs)

[[2.52374159e-03 1.96985076e-01 8.00491182e-01]
 [1.19982149e-02 4.84471646e-01 5.03530139e-01]
 [1.41937404e-03 4.40327581e-01 5.58253045e-01]
 ...
 [1.41937404e-03 4.40327581e-01 5.58253045e-01]
 [7.99688317e-04 4.84704645e-01 5.14495666e-01]
 [4.63652258e-03 4.78172482e-01 5.17190995e-01]]


In [56]:
# Faz previsões de probabilidade
y_pred_probs = rf.predict_proba(X_test)

# Identifica a classe com maior probabilidade
y_pred_indices = np.argmax(y_pred_probs, axis=1)

# Mapeia os índices para as classes de interesse
class_mapping = {0: 0, 1: 1, 2: -1}  # ajuste conforme necessário para sua classificação
y_pred = np.vectorize(class_mapping.get)(y_pred_indices)

# Avalia o modelo
print(classification_report(y_test, y_pred, target_names=["Desfazer Posição", "Compra", "Short"]))

                  precision    recall  f1-score   support

Desfazer Posição       0.01      0.78      0.02         9
          Compra       0.00      0.00      0.00       357
           Short       0.51      0.12      0.19       347

        accuracy                           0.07       713
       macro avg       0.17      0.30      0.07       713
    weighted avg       0.25      0.07      0.09       713



c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier